In [1]:
import numpy as np
import pandas as pd

### 时间区间表示的是时间范围，例如天、月、季度或年。 pandas.Period 类表示此数据类型，需要使用一个字符串或数字以及有效的频率：

In [2]:
# 在这种情况下，Period 对象表示从 2022 年 1 月 1 日到 2022 年 12 月 31 日（含）的完整时间跨度。
p = pd.Period("2022" , freq="A-DEC")
p

Period('2022', 'A-DEC')

### 在时间段上增加或减去整数可以很方便的根据它们的频率进行移位：

In [3]:
p + 3

Period('2025', 'A-DEC')

In [4]:
p - 2

Period('2020', 'A-DEC')

### 如果两个区间具有相同的频率，则它们的差值是它们之间的单位数：

In [5]:
pd.Period("2024", freq="A-DEC") - p

<2 * YearEnds: month=12>

### 可以使用 period_range 函数构造规则的区间序列：

In [6]:
periods = pd.period_range("2022-1-1" , "2022-12-12" , freq="M")
periods

PeriodIndex(['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
             '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12'],
            dtype='period[M]')

### PeriodIndex 类存储的是区间的序列，可以作为任意pandas数据结构中的轴索引：

In [7]:
pd.Series(np.arange(len(periods)), index=periods)

2022-01     0
2022-02     1
2022-03     2
2022-04     3
2022-05     4
2022-06     5
2022-07     6
2022-08     7
2022-09     8
2022-10     9
2022-11    10
2022-12    11
Freq: M, dtype: int32

### 如果你有一个字符串数组，也可以使用 PeriodIndex 类，它的所有值都是时间区间：

In [8]:
values = ["2001Q3", "2002Q2", "2003Q1"]

In [9]:
index = pd.PeriodIndex(values, freq="Q-DEC")
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]')

# 1.区间频率转换：

### Periods 和 PeriodIndex 对象可以使用它们的 asfreq 方法将其转换为另一个频率。

### 例如，假设我们有一个年度区间，并希望在年初或年底将其转换为月度期间。可以这样做：

In [10]:
# Period 对象表示从 2022 年 1 月 1 日到 2022 年 12 月 31 日（含）的完整时间跨度。
p = pd.Period("2022", freq="A-DEC")
p

Period('2022', 'A-DEC')

In [11]:
p.asfreq("M",how="start")

Period('2022-01', 'M')

In [12]:
p.asfreq("M",how="end")

Period('2022-12', 'M')

In [13]:
p.asfreq("M")

Period('2022-12', 'M')

### 对于除了12月份以外的一个月结束的财政年度，相应的每月分期是不同的：

In [14]:
p = pd.Period("2022", freq="A-MAY")
p

Period('2022', 'A-MAY')

In [15]:
p.asfreq("M",how="start")

Period('2021-06', 'M')

In [16]:
p.asfreq("M",how="end")

Period('2022-05', 'M')

### 当你从高频转换为低频时，pandas会根据子区间的所属来决定父区间。

### 例如，在"A-MAY"频率中，"Aug-2021"是2022区间的一部分：

In [17]:
p = pd.Period("Aug-2021", "M")

In [18]:
p.asfreq("A-MAY")

Period('2022', 'A-MAY')

### 可以使用相同的语义转换完整的PeriodIndex 对象或时间序列：

### 例如，使用“M”频率，年度期间将被替换为对应于每个年度区间的第一个月的月度期间。

In [19]:
periods = pd.period_range("2019", "2022", freq="A-DEC")
periods

PeriodIndex(['2019', '2020', '2021', '2022'], dtype='period[A-DEC]')

In [20]:
ts = pd.Series(np.arange(len(periods)), index=periods)
ts

2019    0
2020    1
2021    2
2022    3
Freq: A-DEC, dtype: int32

In [21]:
ts.asfreq("M" , how="start")

2019-01    0
2020-01    1
2021-01    2
2022-01    3
Freq: M, dtype: int32

### 此处，使用“B”频率表明我们想要的区间末端，年度区间将被替换为每年最后一个工作日：

In [22]:
ts.asfreq("B", how="end")

2019-12-31    0
2020-12-31    1
2021-12-31    2
2022-12-30    3
Freq: B, dtype: int32

# 2.季度区间频率：

### 季度数据是会计、金融和其他领域的标准。许多季度数据是相对于财政年度结束报告的，通常是一年中 12 个月之一的最后一个日历日或工作日。

### 因此，2022Q4 期间根据财政年度结束具有不同的含义。 pandas 支持所有 12 种可能的季度频率，从 Q-JAN 到 Q-DEC：

In [23]:
p = pd.Period("2022Q4", freq="Q-JAN")
p

Period('2022Q4', 'Q-JAN')

### 对于在 1 月结束的财政年度，2022Q4 是从 2021 年 11 月到 2022 年 1 月，可以通过转换为每日频率来检查：

In [24]:
p.asfreq("D", how="start")

Period('2021-11-01', 'D')

In [25]:
p.asfreq("D", how="end")

Period('2022-01-31', 'D')

### 因此，可以进行简单的区间运算。

### 例如，获取在本季度的倒数第二个工作日下午 4 点的时间戳：

In [26]:
# T 表示分钟
p4pm = (p.asfreq("B", how="end") - 1).asfreq("T", how="start") + 16 * 60
p4pm

Period('2022-01-28 16:00', 'T')

In [27]:
# to_timestamp 方法默认返回周期开始时的时间戳。
p4pm.to_timestamp()

Timestamp('2022-01-28 16:00:00')

### 可以使用 pandas.period_range 生成季度序列，其算法也是相同的：

In [28]:
periods = pd.period_range("2021Q3", "2022Q4", freq="Q-JAN")
periods

PeriodIndex(['2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4'], dtype='period[Q-JAN]')

In [29]:
ts = pd.Series(np.arange(len(periods)), index=periods)
ts

2021Q3    0
2021Q4    1
2022Q1    2
2022Q2    3
2022Q3    4
2022Q4    5
Freq: Q-JAN, dtype: int32

In [30]:
new_periods = (periods.asfreq("B", "end") - 1).asfreq("H", "start") + 16
new_periods

PeriodIndex(['2020-10-29 16:00', '2021-01-28 16:00', '2021-04-29 16:00',
             '2021-07-29 16:00', '2021-10-28 16:00', '2022-01-28 16:00'],
            dtype='period[H]')

In [31]:
ts.index = new_periods.to_timestamp()

In [32]:
ts

2020-10-29 16:00:00    0
2021-01-28 16:00:00    1
2021-04-29 16:00:00    2
2021-07-29 16:00:00    3
2021-10-28 16:00:00    4
2022-01-28 16:00:00    5
dtype: int32